In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q fastapi

In [ ]:
!pip -q install configparser langchain
!pip -q install transformers huggingface_hub 

In [3]:
import configparser
import os

reader = configparser.ConfigParser()
reader.read_file(open('/content/apidata.config'))

os.environ['SERPAPI_API_KEY']=reader["OPENAI"]["SERPAPI_KEY"]
os.environ['SERPER_API_KEY']=reader["OPENAI"]["SERPER_KEY"]
os.environ['HUGGINGFACEHUB_API_TOKEN'] = reader["OPENAI"]["HFREAD"]

In [ ]:
"""Initiating the Wrapper around HuggingFace Pipeline APIs."""
import importlib.util
from typing import Any, List, Mapping, Optional
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
from pydantic import BaseModel, Extra

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens


In [5]:
DEFAULT_MODEL_ID = "llama-7b-hf"
DEFAULT_TASK = "text-generation"
VALID_TASKS = ("text2text-generation", "text-generation")


In [6]:
class HuggingFacePipeline(LLM, BaseModel):
    """Wrapper around HuggingFace Pipeline API.

    To use, you should have the ``transformers`` python package installed.

    Only supports `text-generation` and `text2text-generation` for now.

    Example using from_model_id:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            hf = HuggingFacePipeline.from_model_id(
                model_id="gpt2", task="text-generation"
            )
    Example passing pipeline in directly:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

            model_id = "gpt2"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(model_id)
            pipe = pipeline(
                "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
            )
            hf = HuggingFacePipeline(pipeline=pipe)
    """

    pipeline: Any  #: :meta private:
    model_id: str = DEFAULT_MODEL_ID
    """Model name to use."""
    model_kwargs: Optional[dict] = None
    """Key word arguments to pass to the model."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    @classmethod
    def from_model_id(
        cls,
        model_id: str,
        task: str,
        device: int = -1,
        model_kwargs: Optional[dict] = None,
        **kwargs: Any,
    ) -> LLM:
        """Construct the pipeline object from model_id and task."""
        try:
            from transformers import (
                LLaMAForCausalLM,
                LLaMATokenizer,
            )
            from transformers import pipeline as hf_pipeline

        except ImportError:
            raise ValueError(
                "Could not import transformers python package. "
                "Please it install it with `pip install transformers`."
            )

        _model_kwargs = model_kwargs or {}
        tokenizer = LLaMATokenizer.from_pretrained(model_id, **_model_kwargs)

        try:
            if task == "text-generation":
                model = LLaMAForCausalLM.from_pretrained(model_id,
                                                         load_in_8bit=True,
                                                         device_map='auto')
            else:
                raise ValueError(
                    f"Got invalid task {task}, "
                    f"currently only {VALID_TASKS} are supported"
                )
        except ImportError as e:
            raise ValueError(
                f"Could not load the {task} model due to missing dependencies."
            ) from e

        if importlib.util.find_spec("torch") is not None:
            import torch

            cuda_device_count = torch.cuda.device_count()
            if device < -1 or (device >= cuda_device_count):
                raise ValueError(
                    f"Got device=={device}, "
                    f"device is required to be within [-1, {cuda_device_count})"
                )
            if device < 0 and cuda_device_count > 0:
                print(
                    "Device has %d GPUs available. "
                    "Provide device={deviceId} to `from_model_id` to use available"
                    "GPUs for execution. deviceId is -1 (default) for CPU and "
                    "can be a positive integer associated with CUDA device id.",
                    cuda_device_count,
                )

        pipeline = hf_pipeline(
            task=task,
            model=model,
            tokenizer=tokenizer,
            device=device,
            model_kwargs=_model_kwargs,
        )
        if pipeline.task not in VALID_TASKS:
            raise ValueError(
                f"Got invalid task {pipeline.task}, "
                f"currently only {VALID_TASKS} are supported"
            )
        return cls(
            pipeline=pipeline,
            model_id=model_id,
            model_kwargs=_model_kwargs,
            **kwargs,
        )

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            **{"model_id": self.model_id},
            **{"model_kwargs": self.model_kwargs},
        }

    @property
    def _llm_type(self) -> str:
        return "huggingface_pipeline"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.pipeline(prompt)
        if self.pipeline.task == "text-generation":
            # Text generation return includes the starter text.
            text = response[0]["generated_text"][len(prompt) :]
        elif self.pipeline.task == "text2text-generation":
            text = response[0]["generated_text"]
        else:
            raise ValueError(
                f"Got invalid task {self.pipeline.task}, "
                f"currently only {VALID_TASKS} are supported"
            )
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [ ]:
#from langchain.llms import HuggingFacePipeline

#hf_fmd = HuggingFacePipeline.from_model_id(model_id="decapoda-research/llama-7b-hf", task="text-generation")

In [7]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

model_id = "decapoda-research/llama-7b-hf"
tokenizer = LLaMATokenizer.from_pretrained(model_id)


In [8]:
model = LLaMAForCausalLM.from_pretrained(model_id,
                                         load_in_8bit=True,
                                         device_map='auto')

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [9]:
lora_model = PeftModel.from_pretrained(model, 
                                         "tloen/alpaca-lora-7b")

In [10]:
# Trying out the prompt templates with the Alpaca Lora
from langchain import PromptTemplate

product_prompt = PromptTemplate(
    input_variables=["question"], 
    template="List me 10 things you know about {question}."
)
product_prompt.format(question='Laptop')

'List me 10 things you know about Laptop.'

In [16]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

In [19]:
lora_pipe = pipeline(
    "text2text-generation", model=lora_model, 
    tokenizer=tokenizer, 
    max_new_tokens=100
)

The model 'PeftModelForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [20]:
lora_model = HuggingFacePipeline(pipeline=lora_pipe)

In [21]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt_lora = PromptTemplate(template=template, input_variables=["question"])

lora_chain = LLMChain(prompt=prompt_lora,llm = lora_model)

In [ ]:
question = "What IPL team won in the year 2019?"

print(lora_chain.run(question))

In [34]:
%%sh

pip -q install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 11.1 MB/s eta 0:00:00


In [35]:
%%sh

pip -q install public_ip
pip -q install uvicorn nest_asyncio

In [15]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn
import public_ip

In [37]:
collab_fastAPI = FastAPI()

In [38]:
@collab_fastAPI.get('/')
def root_api():
    return {"message":"Hi from LLM Server"}

In [39]:
@collab_fastAPI.get("/serve_reply/{question}")
def recieve_completion(question):
  completion = lora_chain.run(question)
  return {"API reply":completion}

In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(collab_fastAPI, port=8000)

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/process.py", line 146, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.9/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Public URL: http://a93a-104-154-102-157.ngrok.io


INFO:     Started server process [1092]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2409:4072:6e8e:1625:43c:efdd:13a9:236d:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4072:6e8e:1625:43c:efdd:13a9:236d:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     125.99.88.198:0 - "GET / HTTP/1.1" 200 OK


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.9/threading.py", line 937, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-39-17ed3584cc86>", line 3, in recieve_completion
    completion = lora_chain.run(question)
  File "/usr/local/lib/python3.9/d

INFO:     2409:4072:6e8e:1625:43c:efdd:13a9:236d:0 - "GET /serve_reply/question%3D%22This%20is%20a%20nice%20way%20to%20work%22 HTTP/1.1" 200 OK
INFO:     2409:4072:6e8e:1625:43c:efdd:13a9:236d:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4072:6e8e:1625:43c:efdd:13a9:236d:0 - "GET / HTTP/1.1" 200 OK


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.9/threading.py", line 937, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-39-17ed3584cc86>", line 3, in recieve_completion
    completion = lora_chain.run(question)
  File "/usr/local/lib/python3.9/d

INFO:     2409:4072:6e8e:1625:43c:efdd:13a9:236d:0 - "GET /serve_reply/question%3D%22What%20IPL%20team%20won%20in%20the%20year%202019?%22 HTTP/1.1" 200 OK
